In [10]:
import os
import subprocess
import pandas as pd

csv_path = 'gh_open_repos.csv'
df = pd.read_csv(csv_path)

In [9]:
df["contains_import"] = 0

clone_dir = "cloned_repos/"
os.makedirs(clone_dir, exist_ok=True)

for index, row in df.iterrows():
  url = row["url"]
  subprocess.run(["git", "clone", "--depth", "1", url + ".git", clone_dir + str(index)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  # print(subprocess.run(["dir", "cloned_repos"], stdout=subprocess.PIPE, stderr=subprocess.PIPE).stdout)
  found = subprocess.run(["grep", "-rE", "import datetime|import whenever|import arrow|import pendulum", clone_dir + str(index)], stdout=subprocess.PIPE, stderr=subprocess.PIPE).stdout
  df.at[index, "contains_import"] = 1 if found else 0
  print(f"{index}: {df.at[index, 'contains_import']}")
  subprocess.run(['rm', '-rf', clone_dir + str(index)])

  if index % 10 == 0:
    output_csv_path = f"repos_with_datetime_check_{index/10}.csv"
    df.to_csv(output_csv_path, index=False)

output_csv_path = "repos_with_datetime_check_final.csv"
df.to_csv(output_csv_path, index=False)

0: 0
1: 0
2: 0


KeyboardInterrupt: 

In [16]:
df = df[df["contains_import"] != 0]

output_csv_path = "repos_with_datetime.csv"
df.to_csv(output_csv_path, index=False)